# AS/RS SIM OPT - Metamodel training and testing

## Import

In [ ]:
import os
import json
import numpy as np
import sys
np.set_printoptions(threshold=sys.maxsize)
import os
np.set_printoptions(suppress=True)
from scipy import stats
import pandas as pd
np.set_printoptions(suppress=True)
from scipy.stats import spearmanr
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
current_directory = os.getcwd()

## Load data

In [ ]:
phase = "metamodel_training_and_testing"
file_name = "1_data" + os.sep + "raw" + os.sep + phase
dir_raw_data = os.path.join(current_directory, file_name)
iterations = os.listdir(dir_raw_data)

In [ ]:
list_json_order_set = []
list_json_racks = []
list_json_mls = []
list_json_failures = []
list_json_capacities = []
list_output = []
num_good_iteration = 0
num_good_replications = 0
total_num_simulation = len(iterations)
max_replications = 1
aggregation_factor = 100

for i,iteration in enumerate(iterations):

    path_iteration = os.path.join(dir_raw_data, iteration)

    # Check if the iteration path is a directory

    if os.path.isdir(path_iteration):

        all_files = os.listdir(path_iteration)
        print(f"Iteration {i+1} / {total_num_simulation}")
        
        #check context data
        files_context = [f for f in all_files if os.path.isfile(os.path.join(path_iteration, f))]
        dir_replications = [f for f in all_files if os.path.isdir(os.path.join(path_iteration, f))][0]

        num_good_iteration += 1

        for file_context in files_context:
            iteration_capacity = None
            path_file_context = os.path.join(path_iteration,file_context)
            with open(path_file_context, "r") as file:
                json_content = json.load(file)
                if file_context == "order_sets.json":
                    iteration_order_set = json_content
                elif file_context == "racks_data.json":
                    iteration_racks_data = json_content
                elif file_context == "mls_data.json":
                    iteration_mls = json_content
                elif file_context == "failure_data.json":
                    iteration_failures = json_content
                elif file_context == "capacity_data.json":
                    iteration_capacity = json_content

        path_dir_replications = os.path.join(path_iteration,dir_replications)
        files_replications = os.listdir(path_dir_replications)
        num_replications = len(files_replications)
        num_good_replications += num_replications
        index_replication = 0

        output_to_aggregate_all = []

        for j in range(num_replications):

            path_replication = os.path.join(path_dir_replications,files_replications[j])

            with open(path_replication, "r") as file:
                json_content = json.load(file)
                if json_content[0][2] == True:
                    output_value_all = [50000,1]
                else:
                    output_value_all = [json_content[0][1],0]

                output_to_aggregate_all.append(output_value_all)

        # sostituisco i valori quando accade deadlock con la media delle altre replicazioni
        deadlock_data_all = [o[0] for o in output_to_aggregate_all if o[0] == 50000]
        not_deadlock_data_all = [o[0] for o in output_to_aggregate_all if o[0] != 50000]
        if deadlock_data_all:
            mean_not_deadlock_all = np.mean(not_deadlock_data_all) # calcola una media generale just in case

        while(num_replications):

            output_to_aggregate = []

            for _ in range(aggregation_factor):

                if (num_replications):
                    output_to_aggregate.append(output_to_aggregate_all[index_replication])

                else: # esce se non bastano le replicazioni
                    break

                num_replications -= 1
                index_replication += 1

            # sostituisco i valori quando accade deadlock con la media delle altre replicazioni
            deadlock_data = [o[0] for o in output_to_aggregate if o[0] == 50000]
            not_deadlock_data = [o[0] for o in output_to_aggregate if o[0] != 50000]

            if deadlock_data:

                if not_deadlock_data:
                    mean_not_deadlock = np.mean(not_deadlock_data) # se ci sono not_deadlock_data sostituisce con la media del cluster
                else:
                    mean_not_deadlock = mean_not_deadlock_all # se non ci sono not_deadlock_data sostituisce con la media generale

                for o in output_to_aggregate:
                    if o[0] == 50000:
                        o[0] = mean_not_deadlock

            aggreation = [np.mean(pair) for pair in list(zip(*output_to_aggregate))]
            list_output.append(aggreation)
            list_json_order_set.append(iteration_order_set)
            list_json_racks.append(iteration_racks_data)
            list_json_mls.append(iteration_mls)
            list_json_failures.append(iteration_failures)
            list_json_capacities.append(iteration_capacity if iteration_capacity is not None else 0)
            
print("Total iterations: ", total_num_simulation, " Good iterations: ", num_good_iteration)
print("Total replications: ", total_num_simulation*max_replications, " Good replications: ", num_good_replications)


In [ ]:
list_output = []
num_good_iteration = 0
num_good_replications = 0
total_num_simulation = len(iterations)
max_replications = 100
aggregation_factor = 1

for i, iteration in enumerate(iterations):

    print(f"Iteration {i+1} / {total_num_simulation}")
    
    path_iteration = os.path.join(dir_raw_data, iteration)

    # Check if the iteration path is a directory

    if os.path.isdir(path_iteration):

        all_files = os.listdir(path_iteration)
        dir_replications = [f for f in all_files if os.path.isdir(os.path.join(path_iteration, f))][0]

        path_dir_replications = os.path.join(path_iteration,dir_replications)
        files_replications = os.listdir(path_dir_replications)
        num_replications = len(files_replications)
        num_good_replications += num_replications
        index_replication = 0

        output_all = []

        for j in range(num_replications):

            path_replication = os.path.join(path_dir_replications,files_replications[j])

            with open(path_replication, "r") as file:
                json_content = json.load(file)
                if json_content[0][2] == True:
                    output_value = 5000
                else:
                    output_value = json_content[0][1]

                output_all.append(output_value)

        output_all = sorted(output_all)
        list_output.append(output_all)

print("Total iterations: ", total_num_simulation, " Good iterations: ", num_good_iteration)
print("Total replications: ", total_num_simulation*max_replications, " Good replications: ", num_good_replications)


## Feature extraction

### Extract features order sets

In [ ]:
list_item_classes = ['A', 'B', 'C']

dict_class = {
    100: 'A',
    250: 'B',
    500: 'C'
}

def get_item_class(item_id):
    for num, item_class in dict_class.items():
        if item_id <= num:
            return item_class

def feature_extraction_orders(order):
    
    # n_prod per class
    # sum, min, avg, max, st_dev product_id per class
    # sum, min, avg, max, st_dev quantity per class

    ord_line_np = np.array(order)
    item_ids = ord_line_np[:, 0]
    item_quantities = ord_line_np[:, 1]
    features = []

    '''
    # n_prod
    features.append(len(order))

    # sum, min, avg, max, st_dev quantity
    features.extend([np.sum(item_quantities), np.min(item_quantities),
                     np.mean(item_quantities), np.max(item_quantities), np.std(item_quantities)])

    # sum, min, avg, max, st_dev product_id                      
    features.extend([np.sum(item_ids), np.min(item_ids),
                     np.mean(item_ids), np.max(item_ids), np.std(item_ids)])

    '''
    
    # n_prod per class
    for item_class in list_item_classes:
        class_mask = np.array([get_item_class(item_id) == item_class for item_id in item_ids])
        features.append(np.sum(class_mask))

    # sum, min, avg, max, st_dev product_id per class
    for item_class in list_item_classes:
        class_ids = item_ids[np.array([get_item_class(item_id) == item_class for item_id in item_ids])]
        if not class_ids.size:
            features.extend([0, 0, 0, 0, 0])
        else:
            features.extend([np.sum(class_ids), np.min(class_ids),
                            np.mean(class_ids), np.max(class_ids), np.std(class_ids)])

    # sum, min, avg, max, st_dev quantity per class
    for item_class in list_item_classes:
        class_quantities = item_quantities[np.array([get_item_class(item_id) == item_class for item_id in item_ids])]
        if not class_quantities.size:
            features.extend([0, 0, 0, 0, 0])
        else:
            features.extend([np.sum(class_quantities), np.min(class_quantities),
                            np.mean(class_quantities), np.max(class_quantities), np.std(class_quantities)])

    return features

In [ ]:
all_features_order_sets = []

for i, ord_set in enumerate(list_json_order_set):

    features_order_set = []

    for j, order in enumerate(ord_set):

        order_lines = [list(map(int, element.split(';'))) for element in order]
        features_order = feature_extraction_orders(np.array(order_lines))
        features_order_set.append(features_order)

    all_features_order_sets.append(features_order_set)

max_n_orders = max([len(order_set) for order_set in all_features_order_sets])
for order_set in all_features_order_sets:
    missing_order = max_n_orders - len(order_set)
    for _ in range(missing_order):
        order_set.extend([[0]*len(order_set[0])])

np_features_order_sets = np.array(all_features_order_sets)
print(np_features_order_sets.shape)

### Extract features racks

In [ ]:
def extract_features_rack(rack):
    
    features = []
    sl_classes = ['A', 'B', 'C']

    for sl_class in sl_classes:
        
        n_boxes = rack[sl_class][0]
        deep_pos = rack[sl_class][1]
        products_ids = rack[sl_class][2]
        quantities = rack[sl_class][3]

        # sum, min, avg, max, st dev for n_boxes, deep_pos, products_ids, and quantities
        '''
        feature_rack = [
            np.sum(n_boxes), np.min(n_boxes), np.mean(n_boxes),
            np.max(n_boxes), np.std(n_boxes),
            np.sum(deep_pos), np.min(deep_pos), np.mean(deep_pos),
            np.max(deep_pos), np.std(deep_pos),
            np.sum(products_ids), np.min(products_ids), np.mean(products_ids),
            np.max(products_ids), np.std(products_ids),
            np.sum(quantities), np.min(quantities), np.mean(quantities),
            np.max(quantities), np.std(quantities)
        ]
        '''

        feature_rack = [
            np.sum(n_boxes),np.sum(deep_pos),
            np.sum(products_ids), np.min(products_ids), np.mean(products_ids),
            np.max(products_ids), np.std(products_ids),
            np.sum(quantities), np.min(quantities), np.mean(quantities),
            np.max(quantities), np.std(quantities)
        ]

        features.extend(feature_rack)

    return features

In [ ]:
all_features_rack = []

for simulation in list_json_racks:
    
    features_rack = []

    for rack in simulation:

        loc_class = {
            'A': [[], [], [], []],
            'B': [[], [], [], []],
            'C': [[], [], [], []]
        }

        for location in rack:
            loc_class[location['sl_class']][0].extend([location['n_boxes']])
            loc_class[location['sl_class']][1].extend([location['deep_occ']])
            loc_class[location['sl_class']][2].extend(location['list_prod'])
            loc_class[location['sl_class']][3].extend(location['list_qnt'])

        features_rack.append(extract_features_rack(loc_class))

    all_features_rack.append(features_rack)

np_all_features_rack = np.array(all_features_rack)
np_all_features_rack = np_all_features_rack.reshape(np_all_features_rack.shape[0],-1)
print(np_all_features_rack.shape)

### Extract features MLS

In [ ]:
all_features_mls = []

for simulation in list_json_mls:
    
    features_mls = []

    features_mls.extend([x for x in simulation[0]])
    features_mls.extend([x for x in simulation[1]])
    all_features_mls.append(features_mls)

np_all_features_mls = np.array(all_features_mls)
print(np_all_features_mls.shape)

### Extract features failures

In [ ]:
all_features_failures = []

for simulation in list_json_failures:
    features_failures = []
    features_failures.extend(simulation)
    all_features_failures.append(features_failures)

np_all_features_failures = np.array(all_features_failures)
print(np_all_features_failures.shape)

bool_mask = np.logical_or(np_all_features_failures == True, np_all_features_failures == False)

# Separate numerical and boolean variables
np_all_features_numerical = np_all_features_failures[:, ~bool_mask.any(axis=0)]
np_all_features_state = np_all_features_failures[:, bool_mask.any(axis=0)]

print("Shape of Numerical Variables Array:", np_all_features_numerical.shape)
print("Shape of Boolean Variables Array:", np_all_features_state.shape)

### Extract capacity

In [ ]:
np_all_features_capacities = np.array(list_json_capacities)
print(np_all_features_capacities.shape)
print(np_all_features_capacities)

### Extract output

In [ ]:
output_np = np.array(list_output)
print(output_np.shape)

## Save data

In [ ]:
np.save(os.path.join(current_directory,"1_data","processed",phase,"features_order_sets.npy"),np_features_order_sets)
np.save(os.path.join(current_directory,"1_data","processed",phase,"features_racks.npy"),np_all_features_rack)
np.save(os.path.join(current_directory,"1_data","processed",phase,"features_mls.npy"),np_all_features_mls)
np.save(os.path.join(current_directory,"1_data","processed",phase,"features_failures.npy"), np_all_features_numerical)
np.save(os.path.join(current_directory,"1_data","processed",phase,"features_state.npy"),np_all_features_state)
np.save(os.path.join(current_directory,"1_data","processed",phase,"features_capacities.npy"),np_all_features_capacities)
np.save(os.path.join(current_directory,"1_data","processed",phase,"output.npy"),output_np)

## Preprocessing

In [ ]:
current_directory = os.getcwd()
directory = os.path.join(current_directory,"1_data","processed",phase)

In [ ]:
path_features_order_sets = os.path.join(directory,"features_order_sets.npy")
features_order_sets_all = np.load(path_features_order_sets)

path_features_racks = os.path.join(directory,"features_racks.npy")
features_racks_all = np.load(path_features_racks)

path_features_mls = os.path.join(directory,"features_mls.npy")
features_mls_all = np.load(path_features_mls)

path_features_failures = os.path.join(directory,"features_failures.npy")
features_failures_all = np.load(path_features_failures)

path_features_state = os.path.join(directory,"features_state.npy")
features_state_all = np.load(path_features_state)

path_output = os.path.join(directory,"output.npy")
output_ann_all = np.load(path_output)

np_multiplier = 1 - features_state_all
features_upstate_all = features_failures_all * np_multiplier
features_failures_all = features_failures_all * features_state_all

print(np_multiplier[0,:])
print(features_upstate_all[0,:])
print(features_failures_all[0,:])

print(f"Order sets -> {features_order_sets_all.shape}")
print(f"Racks -> {features_racks_all.shape}")
print(f"MLS -> {features_mls_all.shape}")
print(f"Upstates -> {features_upstate_all.shape}")
print(f"Failures -> {features_failures_all.shape}")
print(f"Output -> {output_ann_all.shape}")

### Recurrent features

In [ ]:
palette = sns.color_palette('PuBu', as_cmap=True)

new_slided_features_order_sets_all = np.reshape(features_order_sets_all, (features_order_sets_all.shape[0] * features_order_sets_all.shape[1], features_order_sets_all.shape[-1]))
correlation_matrix, _ = spearmanr(new_slided_features_order_sets_all, axis=0)

# Step 3: Plot the correlation matrix
plt.figure(figsize=(8, 6))
plt.imshow(correlation_matrix, cmap=palette, interpolation='nearest')
plt.colorbar(label='Correlation Coefficient')
plt.title('Correlation Matrix')
plt.xlabel('Features')
plt.ylabel('Features')
plt.show()


# features_index = [0,1,2,3,8,13,18,23,28] # n_prod per class, sum product_id per class, sum quantity per class
features_index = [0,1,2,18,23,28] # n_prod per class, sum quantity per class
# features_index = [18,23,28] # sum quantity per class
slided_features_order_sets_all = features_order_sets_all[:,:,features_index]

new_slided_features_order_sets_all = np.reshape(slided_features_order_sets_all, (slided_features_order_sets_all.shape[0] * slided_features_order_sets_all.shape[1], slided_features_order_sets_all.shape[-1]))

# Step 2: Calculate correlation matrix using NumPy
correlation_matrix = np.corrcoef(new_slided_features_order_sets_all[:,:], rowvar=False)
correlation_matrix, _ = spearmanr(new_slided_features_order_sets_all[:,:], axis=0)
print(correlation_matrix.shape)

# Step 3: Plot the correlation matrix
plt.figure(figsize=(8, 6))
plt.imshow(correlation_matrix, cmap=palette, interpolation='nearest')
plt.colorbar(label='Correlation Coefficient')
plt.title('Correlation Matrix')
plt.xlabel('Features')
plt.ylabel('Features')
plt.show()

### Linear features

In [ ]:
features_lin = np.concatenate((features_racks_all,features_mls_all,features_upstate_all,features_failures_all),axis=1)

correlation_matrix, _ = spearmanr(features_lin, axis=0)

# Step 3: Plot the correlation matrix
plt.figure(figsize=(8, 6))
plt.imshow(correlation_matrix, cmap=palette, interpolation='nearest')
plt.colorbar(label='Correlation Coefficient')
plt.title('Correlation Matrix')
plt.xlabel('Features')
plt.ylabel('Features')
plt.show()

In [ ]:
# rack features

#features_index = [0,2,7,12,14,19,24,26,31,36,38,43,48,50,55,60,62,67,72,74,79,84,86,91,96,98,103,108,110,115,120,122,127,132,134,139] # solo n_boxes per class, sum product_id per class, sum quantity per class
features_index = [0,12,24,36,48,60,72,84,96,108,120,132] # solo n_boxes per class
#features_index = [0,1,12,13,24,25,36,37,48,49,60,61,72,73,84,85,96,97,108,109,120,121,132,133] # n_boxes per class, n_deep_pos per class

slided_features_racks_all = features_racks_all[:,features_index]
print(slided_features_racks_all.shape)
print(slided_features_racks_all[[0,],:])
reshaped_array = slided_features_racks_all.reshape(slided_features_racks_all.shape[0], -1, 12)
slided_features_racks_all = np.sum(slided_features_racks_all, axis=1)
slided_features_racks_all = slided_features_racks_all[:, np.newaxis]

print(slided_features_racks_all[[0]])
print(slided_features_racks_all.shape)

In [ ]:
# upstate / failure features
# new_failure = -1 * features_failures_all
new_failure = features_failures_all
new_state = features_upstate_all + new_failure
print(new_state.shape)

### conv aisles and mls

list_features = list(range(20))
features_aisles = new_state[:,list_features]

list_mls = [0,5,10,15]
list_conv_aisles = [i for i in list_features if i not in list_mls]
features_fail_mls = features_aisles[:,list_mls]
features_fail_conv_aisles = features_aisles[:,list_conv_aisles]

sums_conv_aisles = []
for i in range(0, features_fail_conv_aisles.shape[1], 2):
    if i + 1 < features_fail_conv_aisles.shape[1]:
        sums_conv_aisles.append(np.sum(features_fail_conv_aisles[:, i:i + 2], axis=1))

sums_conv_aisles = np.array(sums_conv_aisles)
sums_conv_aisles_inverted = np.transpose(sums_conv_aisles)

### picking stations

features_fail_conv_pick = new_state[:,-8:]

sums_pick = []
for i in range(0, features_fail_conv_pick.shape[1], 2):
    if i + 1 < features_fail_conv_pick.shape[1]:
        sums_pick.append(np.sum(features_fail_conv_pick[:, i:i + 2], axis=1))

sums_pick = np.array(sums_pick)
sums_pick_inverted = np.transpose(sums_pick)

### conveyor loop

list_features = list(range(20,50))
features_conv_loop = new_state[:,list_features]

list_features = list(range(30))
list_to_remove = [0,2,12,14,18]
list_conv_loop = [i for i in list_features if i not in list_to_remove]
features_fail_conv_loop = features_conv_loop[:,list_conv_loop]

features_fail_conv_loop[:, 11] += features_fail_conv_loop[:, 12]
features_fail_conv_loop = np.delete(features_fail_conv_loop, 12, axis=1)

features_fail_conv_loop[:, 22] += features_fail_conv_loop[:, 23]
features_fail_conv_loop = np.delete(features_fail_conv_loop, 23, axis=1)

features_new_state = np.concatenate((features_fail_mls,sums_conv_aisles_inverted,features_fail_conv_loop,sums_pick_inverted),axis=1)
print(features_new_state.shape)

In [ ]:
# concat linear features

#features_rec = features_order_sets_all
features_rec = slided_features_order_sets_all
#features_lin = np.concatenate((features_racks_all,features_upstate_all,features_failures_all),axis=1)
features_lin = np.concatenate((slided_features_racks_all,features_new_state),axis=1)

print(features_rec.shape)
print(features_lin.shape)

In [ ]:
# Step 2: Calculate correlation matrix using NumPy
correlation_matrix = np.corrcoef(features_lin, rowvar=False)
correlation_matrix, _ = spearmanr(features_lin, axis=0)

# Step 3: Plot the correlation matrix
plt.figure(figsize=(8, 6))
plt.imshow(correlation_matrix, cmap='viridis', interpolation='nearest')
plt.colorbar(label='Correlation Coefficient')
plt.title('Correlation Matrix')
plt.xlabel('Features')
plt.ylabel('Features')
plt.show()

In [ ]:
# 3500 è l'indice delle simulazioni senza forzare ordini grandi
threeshold = 3500

features_rec = features_rec[:threeshold,:,:]
features_lin = features_lin[:threeshold,:]
output_ann = output_ann_all[:threeshold,:]

print(features_rec.shape)
print(features_lin.shape)
print(output_ann.shape)

In [ ]:
def plot_hist(data):
    # Plot the histogram
    plt.figure(figsize=(10,5))
    plt.hist(data, bins=30, color='blue', alpha=0.7)
    plt.title('Histogram of NumPy Array')
    plt.xlabel('Value')
    plt.ylabel('Frequency')
    plt.grid(True)
    plt.show()

In [ ]:
plot_hist(output_ann[:,0])

In [ ]:
data_all = output_ann_all[:,0]

Q1 = np.percentile(data_all, 25)
Q3 = np.percentile(data_all, 75)
IQR = Q3 - Q1
cut_off = 1.5
outliers = np.where((data_all < Q1 - cut_off * IQR) | (data_all > Q3 + cut_off * IQR))

print(f"Number of outliers -> {len(outliers[0])}")
print(f"Values of outliers: -> {data_all[outliers]}")

features_rec  = np.delete(features_rec, outliers, axis=0)
features_lin  = np.delete(features_lin, outliers, axis=0)
output_ann = np.delete(output_ann, outliers, axis=0)

print(features_rec.shape)
print(features_lin.shape)
print(output_ann.shape)

In [ ]:
plot_hist(output_ann[:,0])

In [ ]:
x = 0
indices = np.where(output_ann[:,0] > x)

sliced_arr = features_rec[indices]
#print("Sliced array based on indices:", sliced_arr[0,:])

# Function to count rows with all values equal to 0 in a batch
def count_zero_rows(batch):
    return np.sum(np.all(batch == 0, axis=1))

columns_to_sum = [0,1,2]

# List to store results
results = []

# Iterate over batches
for batch_index in range(sliced_arr.shape[0]):
    res = []
    batch = sliced_arr[batch_index]
    batch_size = batch.shape[0]
    
    # Count rows with all values equal to 0 in the batch
    zero_rows_count = count_zero_rows(batch)
    
    # Calculate the length of the batch minus the number of zero rows
    effective_length = batch_size - zero_rows_count
    res.append(effective_length)

    # Calculate the sum along the specified columns
    res.append(np.mean(batch[:, columns_to_sum[0]]))
    res.append(np.mean(batch[:, columns_to_sum[1]]))
    res.append(np.mean(batch[:, columns_to_sum[2]]))

    res.append(output_ann[batch_index,0])

    results.append(res)

df = pd.DataFrame(results, columns=['n order', 'n prod a', 'n prod b', 'n prod c','time'])

# Plot histograms for each column with a bigger plot
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(12, 8))
df['n order'].hist(ax=axes[0, 0], bins=10)
df['n prod a'].hist(ax=axes[0, 1], bins=10)
df['n prod b'].hist(ax=axes[1, 0], bins=10)
df['n prod c'].hist(ax=axes[1, 1], bins=10)

axes[0, 0].set_title('n_order')
axes[0, 1].set_title('n prod a')
axes[1, 0].set_title('n prod b')
axes[1, 1].set_title('n prod c')

fig.suptitle('Histograms of Columns')
plt.show()

# Plot scatter plots against 'time' for each column with a bigger plot
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(12, 8))
df.plot.scatter(x='n order', y='time', ax=axes[0, 0], s=10)
df.plot.scatter(x='n prod a', y='time', ax=axes[0, 1], s=10)
df.plot.scatter(x='n prod b', y='time', ax=axes[1, 0], s=10)
df.plot.scatter(x='n prod c', y='time', ax=axes[1, 1], s=10)

fig.suptitle('Scatter Plots against Time (Inverted Axes)')
plt.show()

### Save data

In [ ]:
X1 = features_rec
X2 = features_lin
Y = output_ann

if phase == "metamodel_training_and_testing":
    
    directory = os.path.join(current_directory,"1_data","processed")
    indexes = np.arange(output_ann.shape[0])
    size_split = int(np.ceil(0.8*len(indexes)))

    np.random.shuffle(indexes)
    training_indexes = np.random.choice(indexes, size=size_split, replace=False)
    testing_indexes = [x for x in indexes if x not in training_indexes]

    dict_indexes = {
        'training' : training_indexes,
        'testing' : testing_indexes
    }

    processes = ['training','testing']

    folders = {
        'training' : directory + os.sep +'metamodel_training',
        'testing' : directory + os.sep +'metamodel_testing'
    }
        
    # Salva i dati split
    for split in ['training', 'testing']:
        idx = dict_indexes[split]
        np.save(os.path.join(folders[split], "features_rec.npy"), X1[idx])
        np.save(os.path.join(folders[split], "features_lin.npy"), X2[idx])
        np.save(os.path.join(folders[split], "output.npy"), Y[idx])

else:
    #folder = folders[process]
    np.save(os.path.join(directory,"features_rec.npy"),X1)
    np.save(os.path.join(directory,"features_lin.npy"),X2)
    np.save(os.path.join(directory,"output.npy"),Y)